In [ ]:
# Insérez ici le numéro de votre équipe
VOTRE_NUMERO_EQUIPE = 0
# modifier cette variable selon que vous contrôler la simulation ou le robot réel
SIM = True

In [ ]:
# Import des paquets requis et configuration du ROS Master
import sys; sys.path.append('/opt/ros/noetic/lib/python3/dist-packages/')
from os import environ
if SIM:
    environ['ROS_MASTER_URI'] = "http://localhost:1137{}/".format(VOTRE_NUMERO_EQUIPE)
else:
    environ['ROS_MASTER_URI'] = "http://cpr-ets05-0X:11311/" # X = votre numéro de Dingo
    environ['ROS_IP'] = "10.196.81.23" # adresse IP de votre station de travail
import rospy
from std_msgs.msg import Float32MultiArray
from jackal_msgs.msg import Drive
from nav_msgs.msg import Odometry
from geometry_msgs.msg import Pose
from sensor_msgs.msg import Imu

In [ ]:
# Création d'un nouveau contenant ROSBAG (en écriture)
import rosbag
test_bag = rosbag.Bag('output.bag', 'w')

In [ ]:
# Créer et démarrer un nouveau noeud
rospy.init_node('dingo_controller', anonymous=True)

In [ ]:
# Position subscriber callback
pose_msg = Pose()
def joint_callback(msg):
    global pose_msg
    pose_msg  = msg.pose.pose

In [ ]:
# UMI subscriber callback
imu_msg = Imu()
def imu_callback(msg):
    global imu_msg
    imu_msg = msg

In [ ]:
# ROS subscribers et publishers
cmd_drive_pub = rospy.Publisher('XXXX/cmd_drive', Drive, queue_size=1)
pose_sub = rospy.Subscriber('XXX', Odometry, joint_callback)
imu_sub = rospy.Subscriber('XXX', Imu, imu_callback)

In [ ]:
# Impression des valeurs UMI et des encodeurs - pour tester
print("Imu data: ")
print("Orientation (quaternion): ", 
    imu_msg.orientation.x, imu_msg.orientation.y, 
    imu_msg.orientation.z, imu_msg.orientation.w
)
print("Angular velocity: ", 
    imu_msg.angular_velocity.x, imu_msg.angular_velocity.y, 
    imu_msg.angular_velocity.z
)
print("Linear acceleration: ", 
    imu_msg.linear_acceleration.x, imu_msg.linear_acceleration.y, 
    imu_msg.linear_acceleration.z
)
print("Position: ", 
    pose_msg.position.x, pose_msg.position.y
)

In [ ]:
# Déterminez les valeurs numériques de vitesse aux roues requises pour touner sur un cercle R=1m
cmd_drive_msg = Drive()
cmd_drive_msg.drivers[0] = 0.0
cmd_drive_msg.drivers[1] = 0.0

# Envoi des commandes au roues par topic ROS
cmd_drive_pub.publish(cmd_drive_msg)

In [ ]:
# Enregistrer la trajectoire pendant 20 secondes
start = float(rospy.Time().now().secs)
rate = rospy.Rate(50)
while (float(rospy.Time().now().secs) - start) < 20:
    test_bag.write('/imu', imu_msg, rospy.Time().now())
    test_bag.write('/chassis_pose', pose_msg, rospy.Time().now())
    rate.sleep()

In [ ]:
# arrêter le robot
cmd_drive_msg = Drive()
cmd_drive_msg.drivers[0] = 0.0
cmd_drive_msg.drivers[1] = 0.0

# Envoi des commandes au roues par topic ROS
cmd_drive_pub.publish(cmd_drive_msg)

In [ ]:
# close the rosbag
test_bag.close()